In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [59]:
train = pd.DataFrame.from_csv('train.tsv', header=0)
#train = train.iloc[:len(train) // 10]
train.head()

,y,year,week,shift,item_id,f1,f2,f3,f4,f5,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
Num,,,,,,,,,,,,,,,,,,,,,
0,123438,2012,52,1,20442076,4915.0,38056.0,40185.0,45733.0,59710.0,...,39423.0,41765.0,52590.0,31452.0,44420.0,41865.0,52705.0,36102.0,44163.0,45239.0
1,58410,2012,52,1,20441997,2230.0,18817.0,20110.0,26368.0,31412.0,...,22830.0,25230.0,27850.0,21390.0,27090.0,23170.0,29705.0,19140.0,22055.0,23200.0
2,163930,2012,52,1,20441990,5695.0,47480.0,47619.0,89708.0,166338.0,...,14930.0,44290.0,46412.0,29320.0,21140.0,28406.0,65056.0,31886.0,48750.0,36520.0
3,53902,2012,52,1,20441989,1995.0,17146.0,20066.0,27070.0,35337.0,...,15120.0,12480.0,19780.0,7990.0,8230.0,10650.0,21920.0,13040.0,9780.0,9630.0
4,105970,2012,52,1,20441988,6515.0,49262.0,50045.0,95167.0,102390.0,...,18872.0,19328.0,37168.0,13570.0,19760.0,20208.0,34745.0,18442.0,24700.0,21793.0


In [60]:
test = pd.DataFrame.from_csv('test.tsv', header=0)
test.head()

,year,week,shift,item_id,f1,f2,f3,f4,f5,f6,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
Num,,,,,,,,,,,,,,,,,,,,,
348622,2015,3,3,20447918,960.0,820.0,1128.0,1801.0,1045.0,1190.0,...,1510.0,580.0,969.0,1635.0,895.0,2140.0,1182.0,1020.0,1293.0,1290.0
348623,2015,3,3,20447902,9086.0,12585.0,11595.0,9685.0,12917.0,11157.0,...,22055.0,14235.0,21195.0,18280.0,18270.0,15851.0,16920.0,18320.0,24116.0,21307.0
348624,2015,3,3,20447732,115087.0,147287.0,176065.0,143105.0,202069.0,216920.0,...,302165.0,162232.0,221622.0,256605.0,240047.0,236630.0,206697.0,245652.0,286179.0,285904.0
348625,2015,3,3,20443951,20900.0,24420.0,27068.0,20460.0,25580.0,27135.0,...,39055.0,14445.0,22450.0,22093.0,31175.0,23355.0,15358.0,18930.0,29643.0,33970.0
348626,2015,3,3,20443944,4430.0,5864.0,3310.0,1853.0,2836.0,2807.0,...,120.0,130.0,60.0,30.0,50.0,20.0,20.0,30.0,0.0,0.0


In [55]:
from sklearn.metrics import make_scorer

X = train.drop('y', axis=1)
y = train['y']

def smape(actual, forecast):
    assert len(actual) == len(forecast)
    n = len(actual)
    return 100 / n * np.sum(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast)))

smape_scorer = make_scorer(smape)

In [61]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from xgboost import XGBRegressor

regr = RandomForestRegressor(n_estimators=100, n_jobs=-1)
#regr = GradientBoostingRegressor(n_estimators=100, max_depth=5)
#regr = XGBRegressor(max_depth=5, n_estimators=100)

tscv = TimeSeriesSplit(n_splits=5)

scores = cross_val_score(regr, X, y, cv=tscv, scoring=smape_scorer)
print(scores, scores.mean())

[ 32.91435221  42.2364694   31.31127448  30.5251747   40.80970112] 35.5593943828


In [63]:
regr.fit(X, y)

submission = pd.read_csv('sample_submission.tsv')
submission['y'] = regr.predict(test)
submission.to_csv('submission.tsv', sep=',', index=False)